<a href="https://colab.research.google.com/github/giancnani/ChatBot-Previsao-do-Tempo/blob/main/Chatbot%20Previsao%20do%20Tempo%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# instalação das bibliotecas

!pip install -q google-generativeai

import json
import re
import requests
import google.generativeai as genai
from datetime import date, datetime  # para importação e ajuste de formato de data


In [ ]:
#Criação chave API local para não compartilhamento externo
import os
os.environ["GEMINI_API_KEY"] = "sua chave api gemini real aqui"

In [ ]:
import os

# carrega api
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if GEMINI_API_KEY is None:
    print("⚠️ AVISO: GEMINI_API_KEY não encontrada nas variáveis de ambiente.")
else:
    genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("models/gemini-2.5-flash") #modelo utilizado pelo gemini


In [ ]:
# treinamento e definições para análise das perguntas

def analise_gemini(pergunta: str) -> dict:
    """
    Interpreta a pergunta usando Gemini e gera uma intent com:
    - cidade
    - tipo_pergunta
    - dia_referencia

    Se a pergunta não for sobre clima/previsão, marca como 'fora_escopo'.
    """ #para focar apenas no tópico desejado

    texto_lower = pergunta.lower()
    palavras_clima = [
        "tempo",
        "clima",
        "previsão",
        "previsao",
        "chuva",
        "chover",
        "ensolarado",
        "nublado",
        "sol",
        "calor",
        "frio",
        "temperatura",
        "ensolarado",
        "graus",
    ] #principais palavras gatilho para entender que o assunto é apropriado

    # se nenhuma palavra relacionada aparecer, trata como fora de escopo e sugere perguntas apropriadas
    if not any(p in texto_lower for p in palavras_clima):
        return {
            "cidade": None,
            "tipo_pergunta": "fora_escopo",
            "dia_referencia": 0
        }

    prompt = f"""
    Você interpreta perguntas em português sobre clima/previsão do tempo.

    Responda APENAS a partir de um JSON válido com os campos:
    {{
      "cidade": "nome da cidade",
      "tipo_pergunta": "clima_atual" ou "previsao" ou "choveu_ontem",
      "dia_referencia": inteiro
    }}

    Regras rápidas:
    - Hoje/agora = 0
    - Amanhã = 1
    - Depois de amanhã = 2
    - Daqui X dias = X
    - Semana que vem = 7
    - Ontem = -1, Anteontem = -2
    - Se não souber a cidade, use null.

    Pergunta: "{pergunta}"
    """

    resposta = model.generate_content(prompt).text.strip()

    # extrair JSON
    start = resposta.find("{")
    end = resposta.rfind("}")
    texto_json = resposta[start:end+1]

    try:
        intent = json.loads(texto_json)
    except Exception:
        intent = {"cidade": None, "tipo_pergunta": "previsao", "dia_referencia": 0}

    # CORREÇÃO 1 — 'daqui X dias' para corrigr problema de identificação de dias pedidos
    match = re.search(r"daqui\s+(\d+)\s+dias", texto_lower)
    if match:
        intent["dia_referencia"] = int(match.group(1))

    # CORREÇÃO 2 — 'semana que vem' - para corrigr problema de relação de datas indiretas
    if "semana que vem" in texto_lower:
        intent["dia_referencia"] = 7

    # CORREÇÃO 3 — datas explícitas: 24/12, 01-01 - para corrigr problema de datas explicitas
    data_match = re.search(r"(\d{1,2})[/-](\d{1,2})(?:[/-](\d{2,4}))?", pergunta)
    if data_match:
        dia = int(data_match.group(1))
        mes = int(data_match.group(2))
        ano_str = data_match.group(3)

        hoje = date.today()
        if ano_str:
            ano = int(ano_str)
            if ano < 100:
                ano += 2000
        else:
            ano = hoje.year
            try:
                data_alvo = date(ano, mes, dia)
            except ValueError:
                data_alvo = None
            else:
                if data_alvo < hoje:
                    ano = hoje.year + 1

        try:
            data_alvo = date(ano, mes, dia)
            delta = (data_alvo - hoje).days
            intent["dia_referencia"] = delta
        except ValueError:
            pass

    return intent


In [ ]:


#  api open-meteo para busca de coordenadas


def buscar_coordenadas(cidade: str):
    url = "https://geocoding-api.open-meteo.com/v1/search"
    params = {"name": cidade, "count": 1, "language": "pt", "format": "json"}

    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
    except Exception:
        return None

    data = resp.json()
    if not data.get("results"):
        return None

    r = data["results"][0]
    return {"latitude": r["latitude"], "longitude": r["longitude"], "nome": r["name"]}



In [ ]:

# api open-meteo para busca de previsao


def buscar_previsao(latitude: float, longitude: float, dias: int = 4):

    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,weathercode",
        "forecast_days": dias,
        "timezone": "America/Sao_Paulo"
    }

    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
    except Exception:
        return None

    daily = resp.json().get("daily", {})
    if not daily:
        return None

    previsoes = []
    for i in range(len(daily["time"])):
        previsoes.append({
            "data": daily["time"][i],
            "temp_max": daily["temperature_2m_max"][i],
            "temp_min": daily["temperature_2m_min"][i],
            "chuva_mm": daily["precipitation_sum"][i],
            "weathercode": daily["weathercode"][i],
        })

    return previsoes



In [ ]:

# weather code (cod. de clima) disponiveis na API


weather_map = {
    0: "Céu limpo",
    1: "Poucas nuvens",
    2: "Parcialmente nublado",
    3: "Nublado",
    45: "Nevoeiro",
    48: "Nevoeiro com gelo",
    51: "Chuvisco fraco",
    53: "Chuvisco moderado",
    55: "Chuvisco forte",
    56: "Chuvisco congelante fraco",
    57: "Chuvisco congelante forte",
    61: "Chuva fraca",
    63: "Chuva moderada",
    65: "Chuva forte",
    66: "Chuva congelante fraca",
    67: "Chuva congelante forte",
    71: "Neve fraca",
    73: "Neve moderada",
    75: "Neve forte",
    77: "Grãos de neve",
    80: "Pancadas fracas",
    81: "Pancadas moderadas",
    82: "Pancadas fortes",
    85: "Pancadas de neve fracas",
    86: "Pancadas de neve fortes",
    95: "Tempestade",
    96: "Tempestade com granizo fraco",
    99: "Tempestade com granizo forte",
}

def descrever_weathercode(code):
    return weather_map.get(code, f"Condição não mapeada (código {code})") #para casos de erro/não encontrados


In [ ]:
#classificação de sensação termica para melhorar resposta (ref. brasil)

def classificar_sensacao_termica(tmin, tmax):
    media = (tmin + tmax) / 2
    if media < 10: return "muito frio"
    if media < 18: return "frio"
    if media < 24: return "ameno"
    if media < 30: return "quente"
    return "muito quente"


In [ ]:
# compilado para gerar resposta final

def gerar_resposta_clima(intent):

    cidade = intent.get("cidade")
    tipo = intent.get("tipo_pergunta")
    dia_ref = intent.get("dia_referencia", 0)

    # perguntas fora de escopo (não são sobre clima/previsão)
    if tipo == "fora_escopo":
        return (
            "🤖 Eu sou um bot especializado em clima e previsão do tempo.\n\n"
            "Posso te ajudar com perguntas do tipo:\n"
            "- Previsão para hoje / amanhã em alguma cidade\n"
            "- Se vai chover em determinado lugar\n"
            "- Clima daqui a alguns dias (dentro do limite da API)\n\n"
            "Se quiser, tente algo como:\n"
            "> Qual a previsão em Diadema amanhã?\n"
            "> Vai chover em Campinas hoje?"
        )

    if not cidade:
        return "❗ Não consegui identificar a cidade. Pode tentar novamente mencionando o nome da cidade?"

    # buscar coordenadas
    coord = buscar_coordenadas(cidade)
    if not coord:
        return f"❗ Não encontrei a cidade '{cidade}'. Pode verificar se o nome está correto?"

    nome = coord["nome"]
    lat, lon = coord["latitude"], coord["longitude"]

    # buscar previsão (no máximo 4 dias)
    dias_pedir = 4  # limitação para api
    previsoes = buscar_previsao(lat, lon, dias_pedir)
    if not previsoes:
        return "❗ Por problemas de servidor, não consegui acessar a previsão agora. Tente novamente mais tarde, por favor."

    limite_dias = len(previsoes)

    # 3) se o usuário pediu além do limite (passado ou +4 dias) não traz dados
    if dia_ref >= limite_dias:
        ultima_data_iso = previsoes[-1]["data"]
        try:
            dt_lim = datetime.strptime(ultima_data_iso, "%Y-%m-%d")
            ultima_data_br = dt_lim.strftime("%d/%m/%Y")
        except Exception:
            ultima_data_br = ultima_data_iso

        return (
            "📅 Limite de previsão ultrapassado\n\n"
            f"Para {nome}, a API de clima que estou usando só fornece previsões confiáveis "
            f"para até {limite_dias} dias à frente e sem dados de histórico, atualmente chegando até a data {ultima_data_br}.\n\n"
            "Como o dia que você pediu está além desse horizonte, não é possível trazer "
            "uma previsão confiável para essa data específica.\n\n"
            "Isso é uma limitação da própria fonte de dados, que segue modelos meteorológicos reais. "
            "Se quiser, posso te ajudar com a previsão para hoje ou para os próximos dias disponíveis. 😊"
        )

    # 4) dentro do limite = resposta normal
    prev = previsoes[dia_ref]

    data_iso = prev["data"]  # modificação do formato de data para padrão Brasil
    try:
        dt = datetime.strptime(data_iso, "%Y-%m-%d")
        data_br = dt.strftime("%d/%m/%Y")
    except Exception:
        data_br = data_iso

    tmin = prev["temp_min"]
    tmax = prev["temp_max"]
    chuva = prev["chuva_mm"]
    weathercode = prev["weathercode"]
    descricao = descrever_weathercode(weathercode)
    sensacao = classificar_sensacao_termica(tmin, tmax)

    # label de tempo (para o usuário)
    if tipo == "clima_atual":
        label = "agora"
    elif dia_ref == 0:
        label = "hoje"
    elif dia_ref == 1:
        label = "amanhã"
    else:
        label = f"na data {data_br}"

    # emoji de condição do tempo
    cond = descricao.lower()
    if "tempestade" in cond:
        emoji_cond = "⛈️"
    elif "chuva" in cond or "pancadas" in cond:
        emoji_cond = "🌧️"
    elif "nevoeiro" in cond or "nublado" in cond:
        emoji_cond = "☁️"
    elif "céu limpo" in cond or "poucas nuvens" in cond:
        emoji_cond = "☀️"
    else:
        emoji_cond = "🌡️"

    # compilado resposta
    cabecalho = f"🤖 ClimaBot – Previsão {label}"
    bloco_local = f"🌍 Cidade: {nome}\n📅 Data da previsão: {data_br}"
    bloco_clima = (
        f"{emoji_cond} Condição geral: {descricao}\n"
        f"🌡️ Temperatura: de {tmin}°C a {tmax}°C\n"
        f"🌧️ Chuva prevista no dia: {chuva} mm\n"
        f"🔎 Sensação térmica aproximada: {sensacao}"
    )

    # comentário e sugestão adaptados de acordo com a resposta para gerar interação mais cpmpleta
    if "chuva" in cond or "tempestade" in cond:
        comentario = "Parece que o dia será marcado por **instabilidade** e bastante umidade."
        sugestao = "Considere atividades em locais fechados e não esqueça o guarda-chuva! ☔"
    elif "céu limpo" in cond or "nuvens" in cond:
        comentario = "O cenário está **bastante favorável**, sem sinais de extremos."
        sugestao = "Um ótimo momento para caminhadas, exercícios ao ar livre ou deslocamentos tranquilos. 🚶‍♀️"
    else:
        comentario = "O clima está **moderado**, sem extremos muito claros."
        sugestao = "Vale acompanhar possíveis mudanças ao longo do dia, principalmente se tiver compromissos ao ar livre."

    fechamento = "ℹ️ Vale acompanhar a previsão ao longo do dia se tiver algo importante marcado."

    resposta_final = (
        f"{cabecalho}\n\n"
        f"{bloco_local}\n\n"
        f"{bloco_clima}\n\n"
        f"💬 {comentario}\n"
        f"💡 {sugestao}\n\n"
        f"{fechamento}"
    )

    return resposta_final


In [ ]:
#bibliotecas para parte visual

from IPython.display import display
import ipywidgets as widgets
import html
from datetime import datetime

# config. visuais

BV_BLUE = "#0046ff"
BG_GREY = "#f5f6fa"
USER_COLOR = "#cfe1ff"
BOT_COLOR = "#ffffff"
TEXT_COLOR = "#333"

USER_ICON = "🙂"
BOT_ICON = "🤖"


chat_messages = []

#config. visuais do bot
def render_chat():
    css = f"""
    <style>
    .chat-wrapper {{
        background: {BG_GREY};
        border-radius: 12px;
        padding: 0;
        max-height: 470px;
        overflow-y: auto;
        border: 1px solid #ddd;
        font-family: Arial, sans-serif;
    }}

    .header {{
        background: {BV_BLUE};
        color: white;
        padding: 12px;
        font-size: 18px;
        font-weight: bold;
        text-align: center;
        border-radius: 12px 12px 0 0;
    }}

    .msg-row {{
        margin: 14px;
        display: flex;
        flex-direction: column;
        max-width: 85%;
    }}

    .user-msg {{
        align-self: flex-end;
        text-align: right;
    }}

    .bot-msg {{
        align-self: flex-start;
        text-align: left;
    }}

    .bubble {{
        padding: 10px 14px;
        border-radius: 14px;
        font-size: 14px;
        line-height: 1.4;
        box-shadow: 0 1px 3px rgba(0,0,0,0.12);
        display: inline-block;
    }}

    .user-bubble {{
        background: {USER_COLOR};
        color: {TEXT_COLOR};
        border-bottom-right-radius: 4px;
    }}

    .bot-bubble {{
        background: {BOT_COLOR};
        color: {TEXT_COLOR};
        border-bottom-left-radius: 4px;
    }}

    .icon-time-row {{
        display: flex;
        align-items: center;
        gap: 6px;
        margin-top: 4px;
        font-size: 11px;
        opacity: 0.8;
    }}

    .icon {{
        font-size: 20px;
    }}

    .time-label {{
        font-size: 11px;
    }}
    </style>
    """

    body = ['<div class="chat-wrapper">']
    body.append(f'<div class="header">ClimaBot – BV</div>')

    if not chat_messages:
        # mensagem inicial do bot
        agora = datetime.now().strftime("%H:%M")
        body.append(
            f"""
            <div class="msg-row bot-msg">
                <div class="bubble bot-bubble">
                    Bem-vindo! Me pergunte algo sobre a previsão do tempo! 😊
                </div>
                <div class="icon-time-row">
                    <div class="icon">{BOT_ICON}</div>
                    <div class="time-label">{agora}</div>
                </div>
            </div>
            """
        )
    else:
        for msg in chat_messages:
            sender = msg["sender"]
            text = html.escape(msg["text"]).replace("\n", "<br>")
            time_str = msg.get("time", "")
            if sender == "user":
                body.append(
                    f"""
                    <div class="msg-row user-msg">
                        <div class="bubble user-bubble">{text}</div>
                        <div class="icon-time-row" style="justify-content: flex-end;">
                            <div class="time-label">{time_str}</div>
                            <div class="icon">{USER_ICON}</div>
                        </div>
                    </div>
                    """
                )
            else:
                body.append(
                    f"""
                    <div class="msg-row bot-msg">
                        <div class="bubble bot-bubble">{text}</div>
                        <div class="icon-time-row">
                            <div class="icon">{BOT_ICON}</div>
                            <div class="time-label">{time_str}</div>
                        </div>
                    </div>
                    """
                )

    # rolar automaticamente pro final
    body.append("""
    <script>
    var cont = document.querySelector('.chat-wrapper');
    if (cont) { cont.scrollTop = cont.scrollHeight; }
    </script>
    """)

    return css + "".join(body)


chat_html = widgets.HTML()


def update_chat():
    chat_html.value = render_chat()


#envio com Enter

def enviar_mensagem(user_text: str):
    user_text = user_text.strip()
    if not user_text:
        return


    # adiciona mensagem do usuário
    chat_messages.append({"sender": "user", "text": user_text})
    update_chat()

    # adiciona mensagem temporária de "digitando..."
    chat_messages.append({"sender": "bot", "text": "🤖 digitando..."})
    update_chat()

    # processa resposta do bot
    try:
        intent = analise_gemini(user_text)
        bot_text = gerar_resposta_clima(intent)
    except Exception as e:
        bot_text = f"⚠️ Erro de sistema ao processar sua pergunta:\n{e}"

    # horário da resposta
    hora_resposta = datetime.now().strftime("%H:%M")

    # substitui a última mensagem (digitando...) pela resposta real
    chat_messages[-1] = {"sender": "bot", "text": bot_text, "time": hora_resposta}
    update_chat()


def on_send_clicked(_):
    user_text = input_box.value
    input_box.value = ""
    enviar_mensagem(user_text)


def on_send_enter(change):
    # disparado quando o usuário aperta Enter no Text
    user_text = change["new"]
    input_box.value = ""
    enviar_mensagem(user_text)


#botão nova conversa / limpa conversa

def on_clear_clicked(_):
    global chat_messages
    chat_messages = []
    update_chat()


# campo de entrada de pergunta
input_box = widgets.Text(
    placeholder="Digite sua pergunta e pressione Enter...Ex: Qual a previsão de hoje para São Paulo?",
    layout=widgets.Layout(width="70%")
)

# enviar com Enter
def on_enter(_):
    texto = input_box.value
    input_box.value = ""
    enviar_mensagem(texto)

input_box.on_submit(on_enter)

# botão Enviar
send_button = widgets.Button(
    description="Enviar",
    button_style="primary",
    icon="paper-plane",
    layout=widgets.Layout(width="12%")
)

def on_send_clicked(_):
    texto = input_box.value
    input_box.value = ""
    enviar_mensagem(texto)

clear_button = widgets.Button(
    description="Nova conversa",
    button_style="warning",
    icon="refresh",
    layout=widgets.Layout(width="15%")
)

send_button.on_click(on_send_clicked)
clear_button.on_click(on_clear_clicked)

ui = widgets.VBox([
    chat_html,
    widgets.HBox([input_box, send_button, clear_button])
])

update_chat()
display(ui)
